# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [27]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


# Scraping the data and loading it into a Pandas dataframe

In [28]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode', 'Borough', 'Neighborhood'] 

# Create dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# load libraries for scraping data
from bs4 import BeautifulSoup
import urllib.request
def make_soup(url):
    wikipage = urllib.request.urlopen(url)
    soupdata = BeautifulSoup(wikipage, "html.parser")
    return soupdata

soup = make_soup("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# find the table
table = soup.find_all('table', class_="wikitable sortable")[0]  

# iterate over it
wikidatasaved = []
for item in table.findAll('tr'):
    wikidata = []
    for data in item.findAll('td'):
        wikidata.append(data.text)
    if len(wikidata) == 3:
        neighborhoods = neighborhoods.append({'Postalcode': wikidata[0],
                                          'Borough': wikidata[1],
                                          'Neighborhood': wikidata[2]}, ignore_index=True)

# remove newline from Neighborhood column
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].map(lambda x: str(x)[:-1])
neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [29]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned'].reset_index(drop = True)
neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [30]:
# More than one neighborhood can exist in one postal code area. Combining them into one row the neighborhoods separated with a comma  
neighborhoods = neighborhoods.groupby([neighborhoods.Postalcode, neighborhoods.Borough], as_index=False).agg(', '.join)
neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
neighborhoods['Neighborhood'].loc[neighborhoods['Neighborhood'] == 'Not assigned'] = neighborhoods['Borough']

In [32]:
neighborhoods.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood

In [33]:
locations = pd.read_csv('http://cocl.us/Geospatial_data')
neighborhoods = neighborhoods.join(locations)
neighborhoods.head(11)

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


In [34]:
neighborhoods.drop('Postal Code', axis=1, inplace=True)
neighborhoods.head(11)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Explore and cluster the neighborhoods in Toronto

In [36]:
# convert an address into latitude and longitude values
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [38]:
# Use Foursquare API to explore and segment neighborhoods
CLIENT_ID = 'REMOVED' # your Foursquare ID
CLIENT_SECRET = 'REMOVED' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Let's take a look at the first postcode in our Toronto dataframe and get its coordinates:
neighborhoods.loc[0, 'Neighborhood']

neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [39]:
# use the Foursquare API to get the top 100 venues that are in Toronto within a radius of 1000 meters from its center coordinates
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results # display results

{'meta': {'code': 200, 'requestId': '5ce6908c9fb6b77580a60b62'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d669cba83865481c948fa53-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'primary': True,
         'shortName': 'Spa'}],
       'id': '4d669cba83865481c948fa53',
       'location': {'address': '8130 Sheppard Ave E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside Ave',
        'distance': 595,
        'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
         'Toronto ON M1B 3W3',
         'Canada'],
        'labeledLatLngs': [{'label': 'd

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON data into a Pandas dataframe

# filter columns 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [41]:
# Exploring Neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [42]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],latitudes=neighborhoods['Latitude'],longitudes=neighborhoods['Longitude'])


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [43]:
# check the size of the dataframe and the top 10 rows:
print(toronto_venues.shape)
toronto_venues.head(10)

(8522, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,"Rouge, Malvern",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
2,"Rouge, Malvern",43.806686,-79.194353,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
3,"Rouge, Malvern",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
4,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
5,"Rouge, Malvern",43.806686,-79.194353,penguin exhibit,43.819435,-79.185959,Zoo Exhibit
6,"Rouge, Malvern",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
7,"Rouge, Malvern",43.806686,-79.194353,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit
8,"Rouge, Malvern",43.806686,-79.194353,Gorilla Exhibit,43.819080,-79.184235,Zoo Exhibit
9,"Rouge, Malvern",43.806686,-79.194353,Orangutan Exhibit,43.818413,-79.182548,Zoo Exhibit


In [44]:
# Check how many venues have been returned for each neighborhood given the radius of 1000 meters and the venue limit of 100:
toronto_venues.groupby('Neighborhood')['Neighborhood'].count()

Neighborhood
Adelaide, King, Richmond                                                                                         100
Agincourt                                                                                                        100
Agincourt North, L'Amoreaux East, Milliken, Steeles East                                                          91
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown     44
Alderwood, Long Branch                                                                                           100
Bathurst Manor, Downsview North, Wilson Heights                                                                   55
Bayview Village                                                                                                   46
Bedford Park, Lawrence Manor East                                                                                100
Berczy Park                                        

In [45]:
# Unique venue categories have been collected in this dataframe:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 324 uniques categories.


# Analyze all neighborhood

In [46]:
# We find out what kind of venues are in each neighborhood and how frequent these categories of venues are in regards to the total of venues.
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape
#group the rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped.shape
# Let's find out what the top 5 most common venues for each neighborhood in Toronto are:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.07
1           Hotel  0.05
2         Theater  0.04
3            Café  0.04
4  Cosmetics Shop  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.12
1         Coffee Shop  0.07
2            Pharmacy  0.05
3   Indian Restaurant  0.03
4       Shopping Mall  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.20
1         Coffee Shop  0.08
2                Park  0.05
3         Pizza Place  0.04
4   Korean Restaurant  0.03


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.18
1  Fast Food Restaurant  0.11
2     Indian Restaurant  0.09
3           Pizza Place  0.09
4         Grocery Store  0.07


----Alderwood, Long Branch----
                  venue  freq
0           Coffee Shop  0.09
1      

In [47]:
# Let's put this knowledge into a new dataframe with each neighborhood and the top 10 most common venues.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Café,Theater,Beer Bar,Pizza Place,Cosmetics Shop,Japanese Restaurant,Art Gallery,Restaurant
1,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Shopping Mall,Indian Restaurant,Sandwich Place,Cantonese Restaurant,Restaurant,Fast Food Restaurant,Bakery
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Park,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Dessert Shop,Bakery,Korean Restaurant,Vietnamese Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Sandwich Place,Park,Beer Store,Fried Chicken Joint,Café
4,"Alderwood, Long Branch",Coffee Shop,Pizza Place,Burger Joint,Fast Food Restaurant,Clothing Store,Pharmacy,Breakfast Spot,Seafood Restaurant,Café,Electronics Store
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Park,Sandwich Place,Pharmacy,Pizza Place,Bank,Convenience Store,Deli / Bodega,Bagel Shop,Restaurant
6,Bayview Village,Chinese Restaurant,Coffee Shop,Park,Bank,Pharmacy,Shopping Mall,Café,Grocery Store,Bakery,Korean Restaurant
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Bakery,Sushi Restaurant,Sandwich Place,Restaurant,Italian Restaurant,Bagel Shop,Café,Bank,Pizza Place
8,Berczy Park,Coffee Shop,Café,Hotel,Park,Restaurant,Gastropub,Italian Restaurant,Japanese Restaurant,Farmers Market,Seafood Restaurant
9,"Birch Cliff, Cliffside West",Coffee Shop,Park,Bank,Grocery Store,Pizza Place,Beer Store,Asian Restaurant,Fast Food Restaurant,Pharmacy,Diner


# Cluster the neighborhoods in Toronto

Let's cluster the neighborhoods in 5 cluster.


In [48]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

toronto_merged = neighborhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Zoo Exhibit,Fast Food Restaurant,Athletics & Sports,Zoo,Pizza Place,Bus Station,Fried Chicken Joint,Caribbean Restaurant,Liquor Store,Other Great Outdoors
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Breakfast Spot,Coffee Shop,Pet Store,Sandwich Place,Ice Cream Shop,Bar,Liquor Store,Beer Store,Supermarket,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Pizza Place,Park,Coffee Shop,Fast Food Restaurant,Breakfast Spot,Storage Facility,Greek Restaurant,Bank,Bar,Liquor Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Fast Food Restaurant,Pizza Place,Park,Discount Store,Furniture / Home Store,Chinese Restaurant,Supermarket,Beer Store,Indian Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Coffee Shop,Clothing Store,Fast Food Restaurant,Gym,Wings Joint,Pharmacy,Restaurant,Indian Restaurant,Sandwich Place,Pizza Place


In [49]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters